## 从定位参数到仅限关键字参数

### 调用函数时使用 * 和 **“展开”可迭代对象，映射到单个参数


In [1]:
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个HTML标签"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' {0}={1}'.format(attr, value) 
                           for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return "\t".join("<{0}{1}>{2}</{3}>".format(name, attr_str, c, name) for c in content)
    else:
        return '<{0}{1} />'.format(name, attr_str)

In [2]:
tag('br')

'<br />'

In [3]:
tag('p', 'hello', id=33)

'<p id=33>hello</p>'

In [4]:
tag('p', 'hello', 'world', cls='sidebar')

'<p class=sidebar>hello</p>\t<p class=sidebar>world</p>'

In [5]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
    'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class=framed src=sunset.jpg title=Sunset Boulevard />'

## 获取关于参数的信息

### 通过函数属性获取
1) 函数对象有个 __defaults__ 属性，它的值是一个元组，里面保存着定位参数和关键字参数的默认值。
2) 仅限关键字参数的默认值在 __kwdefaults__ 属性中。
3) 参数的名称在__code__ 属性中，它的值是一个 code 对象引用，自身也有很多属性

In [6]:
class clip:
    """以一个指定长度附近截断字符串的函数为例"""
    
    def __init__():
        pass
    
    def clip(text, max_len=80):
        """在max_len前面或后面第一个空格处截断文本"""
        end = None
        if len(text > max_len):
            space_before = text.rfind('', 0, max_len)
            if space_before >= 0:
                end = space_before
            else:
                space_after = text.rfind('', max_len)
                if space_after >= 0:
                    end = space_after
            if end is None:
                # 没找到空格
                end = len(text)
            return text[:end].rstrip()

In [7]:
clip_function = clip.clip
clip_function.__defaults__

(80,)

In [8]:
clip_function.__code__

<code object clip at 0x10cf82810, file "<ipython-input-6-1dacd1162770>", line 7>

In [9]:
clip_function.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

### 使用 inspect 模块

inspect.signature 函数返回一个 inspect.Signature 对象，它有一个 parameters属性，这是一个有序映射，把参数名和 inspect.Parameter 对象对应起来,各个 Parameter 属性也有自己的属性

In [10]:
from inspect import signature
sig_clip = signature(clip_function)
str(sig_clip)

'(text, max_len=80)'

In [11]:
for name, param in sig_clip.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


kind 属性的值是 _ParameterKind 类中的 5 个值之一，列举如下:
    - POSITIONAL_OR_KEYWORD
      可以通过定位参数和关键字参数传入的形参（多数 Python 函数的参数属于此类）
    - VAR_POSITIONAL
      定位参数元组
    - VAR_KEYWORD
      关键字参数字典
    - KEYWORD_ONLY
      仅限关键字参数（ Python 3 新增）
    - POSITIONAL_ONLY
      仅限定位参数；目前， Python 声明函数的句法不支持，但是有些使用 C 语言实现且不接受关键字参数的函数（如 divmod）支持